# Path Visualization

## Data Loading

In [ ]:
import os
import re
from math import isnan, nan


def create_route():
    return {
        "__plot_handle": None,
        "Origin_x": nan,
        "Origin_y": nan,
        "Initial_dir": nan,
        "End_dir": nan,
        "Left_right_sign": nan,
        "path": [
            {
                "PathType": nan,
                "TraceType": nan,
                "EntryPoint_x": nan,
                "EntryPoint_y": nan,
                "ExitPoint_x": nan,
                "ExitPoint_y": nan,
                "CenterPoint_x": nan,
                "CenterPoint_y": nan,
                "R": nan,
                "X_data": [nan] * 10,
                "Y_data": [nan] * 10,
            }
            for _i in range(5)
        ],
    }


routes = {
    "front": create_route(),
    "rear": create_route(),
}

ROUTE_PATTERN = re.compile(
    r"""(?x)
        g(?P<route_type>Front|Rear)Route\.
        (?P<key>\w+)\ =\ (?P<value>-?\d*(?:\.\d*)?)
    """
)
PATH_PATTERN = re.compile(
    r"""(?x)
        g(?P<route_type>Front|Rear)Route\.
        Path\[(?P<path_index>\d+)\]\.
        (?P<key>\w+)\ =\ (?P<value>-?\d*(?:\.\d*)?)
    """
)
DATA_PATTERN = re.compile(
    r"""(?x)
        g(?P<route_type>Front|Rear)Route\.
        Path\[(?P<path_index>\d+)\]\.
        (?P<key>\w+)\[(?P<list_index>\d+)\]\ =\ (?P<value>-?\d*(?:\.\d*)?)
    """
)

candidate_log_files = [
    file_name
    for file_name in os.listdir(".")
    if file_name.startswith("tracking_control_") and file_name.endswith(".log")
]
if len(candidate_log_files) != 1:
    raise RuntimeError(
        f"Unexpected log file candidate(s): {candidate_log_files!r}"
    )

with open(candidate_log_files[0], "r") as log_file:
    for line in log_file:
        if "[ INFO]" not in line:
            continue

        line = line.strip()
        escape_seq_len = line.index("[ INFO]")
        if escape_seq_len > 0:
            line = line[escape_seq_len:-escape_seq_len]

        if match := ROUTE_PATTERN.search(line):
            route_type = match.group("route_type").lower()
            path_index = -1
            list_index = -1
            key = match.group("key")
            value = match.group("value")
        elif match := PATH_PATTERN.search(line):
            route_type = match.group("route_type").lower()
            path_index = int(match.group("path_index"))
            list_index = -1
            key = match.group("key")
            value = match.group("value")
        elif match := DATA_PATTERN.search(line):
            route_type = match.group("route_type").lower()
            path_index = int(match.group("path_index"))
            list_index = int(match.group("list_index"))
            key = match.group("key")
            value = match.group("value")
        else:
            continue

        if "." in value:
            value = float(value)
        else:
            value = int(value)

        route = routes[route_type]
        if path_index == -1:
            if key not in route:
                continue
            # ignore duplicate path attributes in log file
            route[key] = value
        else:
            path = route["path"][path_index]
            if key not in path:
                continue
            if list_index == -1:
                if not isnan(path[key]):
                    break
                path[key] = value
            else:
                list_ = path[key]
                if not isnan(list_[list_index]):
                    break
                list_[list_index] = value

print(f"Loaded {candidate_log_files[0]!r}.")

## Visualization

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df_raw = pd.read_csv(
    "tracking_control_node.csv",
    index_col=0,
    parse_dates=["timestamp"],
)
df_plot = df_raw.query("valid == 1")


def visualize(items: list[tuple[str, tuple[str, str]]]) -> None:
    """
    Args:
        items (list[tuple[str, tuple[str, str]]]):
            (route_type, (design_color, actual_color)), ...
    """

    fig = plt.figure(figsize=(5, 5), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    for route_type, colors in items:
        route = routes[route_type]
        plot_kwargs = dict(
            lw=6,
            ms=12,
            color=colors[0],
            alpha=0.5,
            label=f"Design path ({route_type})",
        )
        # draw circle reference first
        for path in route["path"]:
            if path["TraceType"] == 2:  # circle
                theta = np.linspace(0, 2 * np.pi, 100)
                x = path["R"] * np.cos(theta) + path["CenterPoint_x"]
                y = path["R"] * np.sin(theta) + path["CenterPoint_y"]
                ax.plot(x, y, "--", color="gray", alpha=0.5)  # type: ignore
                ax.plot(
                    path["CenterPoint_x"],
                    path["CenterPoint_y"],
                    "+",
                    color="gray",
                    alpha=0.5,
                )
        # draw other paths
        for path in route["path"]:
            match path["TraceType"]:
                case 1:  # straight line
                    route["__plot_handle"] = ax.plot(
                        (path["EntryPoint_x"], path["ExitPoint_x"]),
                        (path["EntryPoint_y"], path["ExitPoint_y"]),
                        ".-",
                        **plot_kwargs,  # type: ignore
                    )[0]
                case 2:  # circle
                    pass
                case 3 | 4:  # arbitrary curve
                    x0 = route["Origin_x"]
                    y0 = route["Origin_y"]
                    x1 = np.array(path["X_data"])
                    y1 = np.array(path["Y_data"])
                    theta = route["Initial_dir"]
                    sign = route["Left_right_sign"]
                    route["__plot_handle"] = ax.plot(
                        x1 * np.cos(theta) - sign * y1 * np.sin(theta) + x0,
                        x1 * np.sin(theta) + sign * y1 * np.cos(theta) + y0,
                        ".-",
                        **plot_kwargs,  # type: ignore
                    )[0]
                case _ as v:
                    raise ValueError(f"unexpected trace type: {v!r}")

    handles_actual = [
        ax.plot(
            df_plot["x_" + route_type],
            df_plot["y_" + route_type],
            ".-",
            ms=2,
            color=colors[1],
            alpha=0.3,
            label=f"Actual path ({route_type})",
        )[0]
        for route_type, colors in items
    ]

    ax.set(
        xlabel="x",
        ylabel="y",
        aspect="equal",
    )
    ax.legend(
        handles=(
            *[routes[route_type]["__plot_handle"] for route_type, _ in items],
            *handles_actual,
        ),
    )
    ax.grid()

### Front & Rear

In [ ]:
visualize([
    ("front", ("red", "pink")),
    ("rear", ("blue", "cyan")),
])

### Front Only

In [ ]:
visualize([
    ("front", ("red", "pink")),
])

### Rear Only

In [ ]:
visualize([
    ("rear", ("blue", "cyan")),
])